<a href="https://colab.research.google.com/github/jmcaussade/Analitica_textual_con_ML_PF/blob/main/Proyecto_Final_Dataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [1]:
!pip install tqdm
!pip install datasets
!pip install langdetect
!pip install num2words
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filena

In [2]:
import pandas as pd
import os
import re
import string
from num2words import num2words
from tqdm import tqdm
from datasets import load_dataset
from langdetect import detect
from transformers import GPT2Tokenizer
from spellchecker import SpellChecker

## Para trabajar con un pedazo del dataset orginal

In [3]:
## SAMPLE DEL DATASET
dataset = load_dataset("somosnlp/recetas-cocina")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/158 [00:00<?, ?B/s]

dataset.csv:   0%|          | 0.00/39.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28238 [00:00<?, ? examples/s]

In [4]:
df = dataset['train'].to_pandas()

# Sample the first 5000 rows from the DataFrame
#df = df.head(5000)

# Show the first few rows of the sampled data
df.head()

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25
3,Sudado de Posta o Muchacho,https://www.mycolombianrecipes.com/es/sudado-d...,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...",86af61e7-e16a-11ed-8914-a96d6180cd25
4,Sopa de Lentejas Colombiana,https://www.mycolombianrecipes.com/es/sopa-de-...,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,86af71bc-e16a-11ed-b130-a96d6180cd25


In [5]:
df

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25
3,Sudado de Posta o Muchacho,https://www.mycolombianrecipes.com/es/sudado-d...,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...",86af61e7-e16a-11ed-8914-a96d6180cd25
4,Sopa de Lentejas Colombiana,https://www.mycolombianrecipes.com/es/sopa-de-...,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,86af71bc-e16a-11ed-b130-a96d6180cd25
...,...,...,...,...,...
28233,Berenjenas rellenas de carne picada y bechamel...,https://www.recetasgratis.net/receta-de-berenj...,4 Berenjenas 2 Zanahorias 1 Cebolla 1 diente d...,1 Empezaremos a preparar estas berenjenas rell...,b0a6454b-a023-44af-a189-92dee8948490
28234,Puré de patatas con queso y huevo,https://www.recetasgratis.net/receta-de-pure-d...,2 Patatas 2 Huevos 100 gramos de Parmesano 2 l...,"1 Cocemos las patatas y, una vez listas, las p...",6e52da68-9496-47de-b85c-7be7fd0b6ac2
28235,Rollitos de calabacín rellenos de jamón y queso,https://www.recetasgratis.net/receta-de-rollit...,½ Calabacín 100 gramos de Jamón cocido 100 gra...,1 Lo primero que debemos hacer es lavar bien e...,22412360-37fe-47de-9401-d3f1e95fe4e7
28236,Calabacín relleno de quinoa,https://www.recetasgratis.net/receta-de-calaba...,4 Calabacines 1 Cebolla 1 Pimiento verde 1 Pim...,1 Lo primero que tenemos que hacer para elabor...,eedfabb6-d14f-46e4-a34f-dc07b9015636


#1) Pre-processing



## Limpieza dataset


In [6]:
df = df.drop(columns=["url", "uuid"])
df

,title,ingredients,steps
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...
...,...,...,...
28233,Berenjenas rellenas de carne picada y bechamel...,4 Berenjenas 2 Zanahorias 1 Cebolla 1 diente d...,1 Empezaremos a preparar estas berenjenas rell...
28234,Puré de patatas con queso y huevo,2 Patatas 2 Huevos 100 gramos de Parmesano 2 l...,"1 Cocemos las patatas y, una vez listas, las p..."
28235,Rollitos de calabacín rellenos de jamón y queso,½ Calabacín 100 gramos de Jamón cocido 100 gra...,1 Lo primero que debemos hacer es lavar bien e...
28236,Calabacín relleno de quinoa,4 Calabacines 1 Cebolla 1 Pimiento verde 1 Pim...,1 Lo primero que tenemos que hacer para elabor...


In [7]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

Missing Values:
title          5597
ingredients       1
steps             0
dtype: int64


In [8]:
# Drop rows where 'title', "ingredients" y "steps" is null
df = df.dropna(subset=['title'])
df = df.dropna(subset=['ingredients'])
df = df.dropna(subset=['steps'])



In [9]:
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 3223


In [10]:
# Drop duplicate rows
df = df.drop_duplicates()

In [11]:
print(df.dtypes)

title          object
ingredients    object
steps          object
dtype: object


In [12]:
df.head(100)

,title,ingredients,steps
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...
...,...,...,...
95,Arroz Atollado de Pescado,2 cucharadas de mantequilla\r\n½ taza de cebol...,Calentar la mantequilla en una olla mediana a ...
96,Arepas de Plátano Maduro,2 plátanos maduros con cascara lavados y corta...,Cocine los plátanos con la cascara en agua dur...
97,Pasta con Salsa de Champiñones,1 libra de pasta penne\r\n¼ taza de aceite de ...,Cocine la pasta en agua hirviendo con sal hast...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,2 pechugas de pollo sin piel\r\n½ cucharadita...,Caliente la parrilla a fuego medio. Coloque el...


In [13]:
df.shape[0]

19417

## Cambiar números a texto

In [14]:
# Function to convert numbers to words in Spanish
def convert_number_to_spanish(text):
    if not isinstance(text, str):
        return text  # Return original value if not a string

    # Split string into list of words
    temp_str = text.split()
    # Initialize an empty list for processed words
    new_string = []

    for word in temp_str:
        # If word is a digit, convert the digit to words in Spanish
        if word.isdigit():
            temp = num2words(int(word), lang='es')
            new_string.append(temp)
        else:
            # Append the word as it is
            new_string.append(word)

    # Join the words in new_string to form a single string
    return ' '.join(new_string)

In [15]:
# Apply the function to the 'steps' column
df['converted_steps'] = df['steps'].apply(convert_number_to_spanish)
df['converted_ingredients'] = df['ingredients'].apply(convert_number_to_spanish)


In [16]:
df.head(100)

,title,ingredients,steps,converted_steps,converted_ingredients
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...","Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...","En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...","Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...","En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...
...,...,...,...,...,...
95,Arroz Atollado de Pescado,2 cucharadas de mantequilla\r\n½ taza de cebol...,Calentar la mantequilla en una olla mediana a ...,Calentar la mantequilla en una olla mediana a ...,dos cucharadas de mantequilla ½ taza de ceboll...
96,Arepas de Plátano Maduro,2 plátanos maduros con cascara lavados y corta...,Cocine los plátanos con la cascara en agua dur...,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...
97,Pasta con Salsa de Champiñones,1 libra de pasta penne\r\n¼ taza de aceite de ...,Cocine la pasta en agua hirviendo con sal hast...,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne ¼ taza de aceite de o...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,2 pechugas de pollo sin piel\r\n½ cucharadita...,Caliente la parrilla a fuego medio. Coloque el...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel ½ cucharadita d...


## Cambiar fracciones a texto

In [17]:
# Mapping of Unicode fractions to words
unicode_fraction_map = {
    '¼': 'un cuarto',
    '½': 'un medio',
    '¾': 'tres cuartos',
    '⅓': 'un tercio',
    '⅔': 'dos tercios',
    '⅕': 'un quinto',
    '⅖': 'dos quintos',
    '⅗': 'tres quintos',
    '⅘': 'cuatro quintos',
    '⅙': 'un sexto',
    '⅚': 'cinco sextos',
    '⅛': 'un octavo',
    '⅜': 'tres octavos',
    '⅝': 'cinco octavos',
    '⅞': 'siete octavos',
}

# Function to replace fractions in text
def convert_unicode_fractions(text):
    if not isinstance(text, str):
        return text  # Skip non-string inputs

    # Replace each Unicode fraction with its text equivalent
    for unicode_fraction, word in unicode_fraction_map.items():
        text = text.replace(unicode_fraction, word)

    return text


In [18]:
# Apply the function to the 'steps' column
df['converted_steps_2'] = df['converted_steps'].apply(convert_unicode_fractions)

df['converted_ingredients_2'] = df['converted_ingredients'].apply(convert_unicode_fractions)

In [19]:
df.head(100)

,title,ingredients,steps,converted_steps,converted_ingredients,converted_steps_2,converted_ingredients_2
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...","Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...,"Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...","En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...,"En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...","Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...,"Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...","En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ...","En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...
...,...,...,...,...,...,...,...
95,Arroz Atollado de Pescado,2 cucharadas de mantequilla\r\n½ taza de cebol...,Calentar la mantequilla en una olla mediana a ...,Calentar la mantequilla en una olla mediana a ...,dos cucharadas de mantequilla ½ taza de ceboll...,Calentar la mantequilla en una olla mediana a ...,dos cucharadas de mantequilla un medio taza de...
96,Arepas de Plátano Maduro,2 plátanos maduros con cascara lavados y corta...,Cocine los plátanos con la cascara en agua dur...,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...
97,Pasta con Salsa de Champiñones,1 libra de pasta penne\r\n¼ taza de aceite de ...,Cocine la pasta en agua hirviendo con sal hast...,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne ¼ taza de aceite de o...,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne un cuarto taza de ace...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,2 pechugas de pollo sin piel\r\n½ cucharadita...,Caliente la parrilla a fuego medio. Coloque el...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel ½ cucharadita d...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel un medio cuchar...


In [20]:
df = df.drop(columns=["steps", "ingredients", "converted_steps", "converted_ingredients"])

In [21]:
df.head(100)

,title,converted_steps_2,converted_ingredients_2
0,Arepas de Queso,"Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...
1,Sudado de Pollo,"En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...
2,Sancocho Trifásico,"Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...
3,Sudado de Posta o Muchacho,"En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ..."
4,Sopa de Lentejas Colombiana,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...
...,...,...,...
95,Arroz Atollado de Pescado,Calentar la mantequilla en una olla mediana a ...,dos cucharadas de mantequilla un medio taza de...
96,Arepas de Plátano Maduro,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...
97,Pasta con Salsa de Champiñones,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne un cuarto taza de ace...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel un medio cuchar...


## Cambiar todo a español

In [22]:
# Function to detect the language of a given text
def detect_language(text):
    try:
        return detect(text)  # Detects the language and returns a language code (e.g., 'es' for Spanish)
    except:
        return None  # Return None if language detection fails

In [23]:
# Apply the language detection function to the 'steps' column
df['language'] = df['converted_steps_2'].apply(detect_language)

# Check for rows where the language is not Spanish ('es')
non_spanish_rows = df[df['language'] != 'es']

# Display rows in non-Spanish languages
print(non_spanish_rows)



                                          title  \
23                Sancocho de Gallina o Valluno   
8218                 Ceviche de Camarones facil   
8233            Enchiladas Morelianas Mexicanas   
8301               verdaderas enchiladas suizas   
8314             Croquetas de bacalao y cebolla   
...                                         ...   
27274                           PASTEL DE ELOTE   
27625                       Caviar de Berenjena   
27633            Pimientos Rellenos de Quesitos   
27658  Tarta de puerros panceta y queso Gruyere   
27981                       Fresh broccoli soup   

                                       converted_steps_2  \
23     In a large pot, place the chicken, corn, aliño...   
8218   uno SOFRIE LOS CAMARONES EN UN SARTEN ,PICA LA...   
8233   uno PONES A COCER PECHUGAS CON UN PEDAZO DE CE...   
8301   uno SE LE QUITA LA SEMILLA AL CHILE Y LA COLA ...   
8314   uno EN UN CALDERO SOFREIR LA CEBOLLA CON LA MA...   
...                        

In [24]:
#Drop las rows que no estàn en español
df = df[df['language'] == 'es']

In [25]:
# Count the number of rows where the language is not 'es' (not Spanish)
non_spanish_count = (df['language'] != 'es').sum()

# Print the result
print(f"Number of rows with a language different from Spanish: {non_spanish_count}")


Number of rows with a language different from Spanish: 0


## Check misspelled words


In [26]:
#!pip install tqdm

In [27]:
# from tqdm.notebook import tqdm
# tqdm.pandas()

In [28]:
# # Initialize the spell checker for Spanish
# spell = SpellChecker(language='es')

# # Function to correct misspellings while handling capitalization
# def correct_misspellings_with_case(text):
#     if not isinstance(text, str):
#         return text

#     words = text.split()
#     corrected_words = []

#     for word in words:
#         # Preserve capitalization if the word is capitalized
#         is_capitalized = word[0].isupper()
#         corrected_word = spell.correction(word.lower())  # Lowercase for checking

#         # If correction is found, apply capitalization if needed
#         if corrected_word:
#             if is_capitalized:
#                 corrected_word = corrected_word.capitalize()  # Restore capitalization
#             corrected_words.append(corrected_word)
#         else:
#             corrected_words.append(word)  # Use original word if no correction

#     return ' '.join(corrected_words)



In [29]:
# !pip install openpyxl
# !pip install xlsxwriter

In [30]:
# # Apply the function to your dataset
# df['corrected_steps'] = df['converted_steps_2'].apply(correct_misspellings_with_case)
# df['corrected_ingredients'] = df['converted_ingredients_2'].apply(correct_misspellings_with_case)
# df['corrected_title'] = df['title'].apply(correct_misspellings_with_case)

# df.to_excel('ortografia_corregida.xlsx', index=False)

## To lower case

In [31]:
df['corrected_title'] = df['title'].str.lower()

df['corrected_ingredients'] = df['converted_ingredients_2'].str.lower()

df['corrected_steps'] = df['converted_steps_2'].str.lower()

<ipython-input-31-1056a9bab1aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['corrected_title'] = df['title'].str.lower()


## Deshacerse de caracteres especiales

In [32]:
# # Function to remove punctuation and special characters
# def remove_punctuation(text):
#     if not isinstance(text, str):
#         return text  # Return original value if not a string
#     return text.translate(str.maketrans('', '', string.punctuation))

In [33]:
# # Use .loc to apply the function to the 'converted_steps_2' column
# df.loc[:, 'corrected_title'] = df['corrected_title'].apply(remove_punctuation)
# df.loc[:, 'corrected_steps'] = df['corrected_steps'].apply(remove_punctuation)
# df.loc[:, 'corrected_ingredients'] = df['corrected_ingredients'].apply(remove_punctuation)


In [34]:
df.head(100)

,title,converted_steps_2,converted_ingredients_2,language,corrected_title,corrected_ingredients,corrected_steps
0,Arepas de Queso,"Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...,es,arepas de queso,uno taza de harina de arepa blanca o amarilla ...,"combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,"En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...,es,sudado de pollo,ocho muslos de pollo sin la piel uno cucharada...,"en una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,"Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...,es,sancocho trifásico,uno taza de cebolla picada uno pimientón rojo ...,"coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ...",es,sudado de posta o muchacho,"uno posta o muchacho de tres a cuatro libras, ...","en una olla grande ponga la carne, el caldo, e..."
4,Sopa de Lentejas Colombiana,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...,es,sopa de lentejas colombiana,uno cucharadita de aceite vegetal uno taza de ...,cocinar el chorizo en el aceite en una olla gr...
...,...,...,...,...,...,...,...
96,Arepas de Plátano Maduro,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...,es,arepas de plátano maduro,dos plátanos maduros con cascara lavados y cor...,cocine los plátanos con la cascara en agua dur...
97,Pasta con Salsa de Champiñones,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne un cuarto taza de ace...,es,pasta con salsa de champiñones,uno libra de pasta penne un cuarto taza de ace...,cocine la pasta en agua hirviendo con sal hast...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel un medio cuchar...,es,ensalada de pollo a la plancha con aderezo de ...,dos pechugas de pollo sin piel un medio cuchar...,caliente la parrilla a fuego medio. coloque el...
99,Tortilla Paisa o Antioqueña,"En una sartén antiadherente grande, caliente u...",tres cucharadas de aceite vegetal cuatro huevo...,es,tortilla paisa o antioqueña,tres cucharadas de aceite vegetal cuatro huevo...,"en una sartén antiadherente grande, caliente u..."


## Concatenar columnas

In [35]:
# Concatenate the columns and create a new column in the DataFrame
df['combined'] = "Titulo:" + " " + df['corrected_title'] + " " + "Ingredientes:"+ " " + df['corrected_ingredients'] + " " + "Receta:" + " " + df['corrected_steps']

In [36]:
df.head(100)

,title,converted_steps_2,converted_ingredients_2,language,corrected_title,corrected_ingredients,corrected_steps,combined
0,Arepas de Queso,"Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...,es,arepas de queso,uno taza de harina de arepa blanca o amarilla ...,"combine la harina de maíz, agua caliente, el q...",Titulo: arepas de queso Ingredientes: uno taza...
1,Sudado de Pollo,"En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...,es,sudado de pollo,ocho muslos de pollo sin la piel uno cucharada...,"en una olla grande, caliente el aceite vegetal...",Titulo: sudado de pollo Ingredientes: ocho mus...
2,Sancocho Trifásico,"Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...,es,sancocho trifásico,uno taza de cebolla picada uno pimientón rojo ...,"coloque la cebolla, el pimientón, el ajo y el ...",Titulo: sancocho trifásico Ingredientes: uno t...
3,Sudado de Posta o Muchacho,"En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ...",es,sudado de posta o muchacho,"uno posta o muchacho de tres a cuatro libras, ...","en una olla grande ponga la carne, el caldo, e...",Titulo: sudado de posta o muchacho Ingrediente...
4,Sopa de Lentejas Colombiana,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...,es,sopa de lentejas colombiana,uno cucharadita de aceite vegetal uno taza de ...,cocinar el chorizo en el aceite en una olla gr...,Titulo: sopa de lentejas colombiana Ingredient...
...,...,...,...,...,...,...,...,...
96,Arepas de Plátano Maduro,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...,es,arepas de plátano maduro,dos plátanos maduros con cascara lavados y cor...,cocine los plátanos con la cascara en agua dur...,Titulo: arepas de plátano maduro Ingredientes:...
97,Pasta con Salsa de Champiñones,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne un cuarto taza de ace...,es,pasta con salsa de champiñones,uno libra de pasta penne un cuarto taza de ace...,cocine la pasta en agua hirviendo con sal hast...,Titulo: pasta con salsa de champiñones Ingredi...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel un medio cuchar...,es,ensalada de pollo a la plancha con aderezo de ...,dos pechugas de pollo sin piel un medio cuchar...,caliente la parrilla a fuego medio. coloque el...,Titulo: ensalada de pollo a la plancha con ade...
99,Tortilla Paisa o Antioqueña,"En una sartén antiadherente grande, caliente u...",tres cucharadas de aceite vegetal cuatro huevo...,es,tortilla paisa o antioqueña,tres cucharadas de aceite vegetal cuatro huevo...,"en una sartén antiadherente grande, caliente u...",Titulo: tortilla paisa o antioqueña Ingredient...


## Contar numero de palabras

In [37]:
# Iterate through each row in the DataFrame and add word counts to the new columns
df.loc[:, 'steps_word_count'] = df['corrected_steps'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df.loc[:, 'ingredients_word_count'] = df['corrected_ingredients'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df.loc[:, 'combined_word_count'] = df['combined'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)


# Now the DataFrame has new columns with the word counts for each row
# You can calculate the min, max, and average for these new columns

# For 'converted_steps_2'
steps_min = df['steps_word_count'].min()
steps_max = df['steps_word_count'].max()
steps_avg = df['steps_word_count'].mean()

# For 'ingredients'
ingredientes_min = df['ingredients_word_count'].min()
ingredientes_max = df['ingredients_word_count'].max()
ingredientes_avg = df['ingredients_word_count'].mean()

# For 'combined'
combined_min = df['combined_word_count'].min()
combined_max = df['combined_word_count'].max()
combined_avg = df['combined_word_count'].mean()

# Print the results
print(f"Converted Steps 2 - Min: {steps_min}, Max: {steps_max}, Average: {steps_avg:.2f}")
print(f"Converted Ingredientes 2- Min: {ingredientes_min}, Max: {ingredientes_max}, Average: {ingredientes_avg:.2f}")
print(f"Combined - Min: {combined_min}, Max: {combined_max}, Average: {combined_avg:.2f}")





Converted Steps 2 - Min: 6, Max: 1813, Average: 201.17
Converted Ingredientes 2- Min: 2, Max: 209, Average: 40.98
Combined - Min: 22, Max: 1880, Average: 249.60


In [38]:
df.head(100)

,title,converted_steps_2,converted_ingredients_2,language,corrected_title,corrected_ingredients,corrected_steps,combined,steps_word_count,ingredients_word_count,combined_word_count
0,Arepas de Queso,"Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...,es,arepas de queso,uno taza de harina de arepa blanca o amarilla ...,"combine la harina de maíz, agua caliente, el q...",Titulo: arepas de queso Ingredientes: uno taza...,92,29,127
1,Sudado de Pollo,"En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...,es,sudado de pollo,ocho muslos de pollo sin la piel uno cucharada...,"en una olla grande, caliente el aceite vegetal...",Titulo: sudado de pollo Ingredientes: ocho mus...,92,84,182
2,Sancocho Trifásico,"Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...,es,sancocho trifásico,uno taza de cebolla picada uno pimientón rojo ...,"coloque la cebolla, el pimientón, el ajo y el ...",Titulo: sancocho trifásico Ingredientes: uno t...,103,114,222
3,Sudado de Posta o Muchacho,"En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ...",es,sudado de posta o muchacho,"uno posta o muchacho de tres a cuatro libras, ...","en una olla grande ponga la carne, el caldo, e...",Titulo: sudado de posta o muchacho Ingrediente...,85,72,165
4,Sopa de Lentejas Colombiana,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...,es,sopa de lentejas colombiana,uno cucharadita de aceite vegetal uno taza de ...,cocinar el chorizo en el aceite en una olla gr...,Titulo: sopa de lentejas colombiana Ingredient...,129,74,210
...,...,...,...,...,...,...,...,...,...,...,...
96,Arepas de Plátano Maduro,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...,es,arepas de plátano maduro,dos plátanos maduros con cascara lavados y cor...,cocine los plátanos con la cascara en agua dur...,Titulo: arepas de plátano maduro Ingredientes:...,96,55,158
97,Pasta con Salsa de Champiñones,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne un cuarto taza de ace...,es,pasta con salsa de champiñones,uno libra de pasta penne un cuarto taza de ace...,cocine la pasta en agua hirviendo con sal hast...,Titulo: pasta con salsa de champiñones Ingredi...,147,65,220
98,Ensalada de Pollo a la Plancha con Aderezo de ...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel un medio cuchar...,es,ensalada de pollo a la plancha con aderezo de ...,dos pechugas de pollo sin piel un medio cuchar...,caliente la parrilla a fuego medio. coloque el...,Titulo: ensalada de pollo a la plancha con ade...,129,100,244
99,Tortilla Paisa o Antioqueña,"En una sartén antiadherente grande, caliente u...",tres cucharadas de aceite vegetal cuatro huevo...,es,tortilla paisa o antioqueña,tres cucharadas de aceite vegetal cuatro huevo...,"en una sartén antiadherente grande, caliente u...",Titulo: tortilla paisa o antioqueña Ingredient...,133,31,171


In [39]:
df = df.drop(columns=["title", "converted_steps_2", "converted_ingredients_2", "language"])

In [40]:
# Count rows where 'steps_word_count' is greater than 400
count = df[df['combined_word_count'] < 300].shape[0]

# Print the result
print(f"Number of rows with 'steps_word_count' greater than 400: {count}")


Number of rows with 'steps_word_count' greater than 400: 13517


#2) Tokenization

In [44]:
from transformers import GPT2TokenizerFast
from tqdm import tqdm
# Load the Spanish GPT-2 tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("flax-community/gpt-2-spanish")

# Enable tqdm for pandas
tqdm.pandas(desc="Tokenizing data")

# Check if the tokenizer is loaded properly
print(tokenizer)


GPT2TokenizerFast(name_or_path='flax-community/gpt-2-spanish', vocab_size=50256, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [45]:
df['combined_tokens'] = df['combined'].progress_apply(lambda x: tokenizer.encode(x, add_special_tokens=True))


Tokenizing data: 100%|██████████| 19271/19271 [00:30<00:00, 636.03it/s]


In [46]:
# Add a column to store the length of tokenized sequences
df['combined_tokens_length'] = df['combined_tokens'].apply(len)

# Generate summary statistics for token lengths
print("Summary statistics for 'combined_tokens_length':")
print(df['combined_tokens_length'].describe())

# Display the distribution of token lengths
print("\nDistribution of token lengths:")
print(df['combined_tokens_length'].value_counts().sort_index())

# Identify rows with the longest and shortest tokenized sequences
max_length = df['combined_tokens_length'].max()
min_length = df['combined_tokens_length'].min()

longest_rows = df[df['combined_tokens_length'] == max_length]
shortest_rows = df[df['combined_tokens_length'] == min_length]

print("\nRows with the longest tokenized sequences:")
print(longest_rows[['combined', 'combined_tokens']])

print("\nRows with the shortest tokenized sequences:")
print(shortest_rows[['combined', 'combined_tokens']])


Summary statistics for 'combined_tokens_length':
count    19271.000000
mean       364.558819
std        179.332454
min         36.000000
25%        235.000000
50%        342.000000
75%        463.000000
max       2667.000000
Name: combined_tokens_length, dtype: float64

Distribution of token lengths:
combined_tokens_length
36      1
38      1
41      2
45      1
48      2
       ..
1821    1
1921    1
2443    1
2660    1
2667    1
Name: count, Length: 991, dtype: int64

Rows with the longest tokenized sequences:
                                                combined  \
21492  Titulo: ¡un postre delicioso! Ingredientes: tr...   

                                         combined_tokens  
21492  [33025, 1335, 30, 1473, 286, 14802, 14857, 5, ...  

Rows with the shortest tokenized sequences:
                                                combined  \
15851  Titulo: ensalada tortrix Ingredientes: lechuga...   

                                         combined_tokens  
15851  [33025, 133

In [47]:
df.head(100)

,corrected_title,corrected_ingredients,corrected_steps,combined,steps_word_count,ingredients_word_count,combined_word_count,combined_tokens,combined_tokens_length
0,arepas de queso,uno taza de harina de arepa blanca o amarilla ...,"combine la harina de maíz, agua caliente, el q...",Titulo: arepas de queso Ingredientes: uno taza...,92,29,127,"[33025, 1335, 30, 512, 576, 270, 225, 72, 73, ...",184
1,sudado de pollo,ocho muslos de pollo sin la piel uno cucharada...,"en una olla grande, caliente el aceite vegetal...",Titulo: sudado de pollo Ingredientes: ocho mus...,92,84,182,"[33025, 1335, 30, 323, 2254, 225, 72, 73, 1138...",270
2,sancocho trifásico,uno taza de cebolla picada uno pimientón rojo ...,"coloque la cebolla, el pimientón, el ajo y el ...",Titulo: sancocho trifásico Ingredientes: uno t...,103,114,222,"[33025, 1335, 30, 1714, 71, 12639, 952, 538, 1...",336
3,sudado de posta o muchacho,"uno posta o muchacho de tres a cuatro libras, ...","en una olla grande ponga la carne, el caldo, e...",Titulo: sudado de posta o muchacho Ingrediente...,85,72,165,"[33025, 1335, 30, 323, 2254, 225, 72, 73, 643,...",240
4,sopa de lentejas colombiana,uno cucharadita de aceite vegetal uno taza de ...,cocinar el chorizo en el aceite en una olla gr...,Titulo: sopa de lentejas colombiana Ingredient...,129,74,210,"[33025, 1335, 30, 20782, 225, 72, 73, 39829, 1...",295
...,...,...,...,...,...,...,...,...,...
96,arepas de plátano maduro,dos plátanos maduros con cascara lavados y cor...,cocine los plátanos con la cascara en agua dur...,Titulo: arepas de plátano maduro Ingredientes:...,96,55,158,"[33025, 1335, 30, 512, 576, 270, 225, 72, 73, ...",224
97,pasta con salsa de champiñones,uno libra de pasta penne un cuarto taza de ace...,cocine la pasta en agua hirviendo con sal hast...,Titulo: pasta con salsa de champiñones Ingredi...,147,65,220,"[33025, 1335, 30, 10306, 303, 9786, 225, 72, 7...",329
98,ensalada de pollo a la plancha con aderezo de ...,dos pechugas de pollo sin piel un medio cuchar...,caliente la parrilla a fuego medio. coloque el...,Titulo: ensalada de pollo a la plancha con ade...,129,100,244,"[33025, 1335, 30, 17960, 225, 72, 73, 11380, 2...",358
99,tortilla paisa o antioqueña,tres cucharadas de aceite vegetal cuatro huevo...,"en una sartén antiadherente grande, caliente u...",Titulo: tortilla paisa o antioqueña Ingredient...,133,31,171,"[33025, 1335, 30, 28784, 653, 2345, 320, 14479...",232


In [48]:
# Count rows where the length of 'combined_tokens' exceeds 512
rows_with_more_than_512_tokens = df[df['combined_tokens'].apply(len) > 512]

# Get the count of such rows
count = rows_with_more_than_512_tokens.shape[0]

print(f"Number of rows with more than 512 tokens: {count}")


Number of rows with more than 512 tokens: 3478


In [49]:
# Filter out rows with more than 512 tokens
df_filtered = df[df['combined_tokens'].apply(len) <= 512]

# Print the new size of the DataFrame
print(f"Original dataset size: {df.shape[0]}")
print(f"Filtered dataset size: {df_filtered.shape[0]}")


Original dataset size: 19271
Filtered dataset size: 15793


### Check For Null Values

In [50]:
# Check for null values in the 'combined_tokens' column
null_values_combined_tokens = df_filtered['combined_tokens'].isnull().sum()

# Print the result
print(f"Number of null values in 'combined_tokens': {null_values_combined_tokens}")


Number of null values in 'combined_tokens': 0


### Check for Nan values


In [51]:
# Check for NaN values in the 'combined_tokens' column
nan_values_combined_tokens = df_filtered['combined_tokens'].isna().sum()

# Print the result
print(f"Number of NaN values in 'combined_tokens': {nan_values_combined_tokens}")


Number of NaN values in 'combined_tokens': 0


In [52]:
# Display the first few rows of the 'combined_tokens' column
print(df_filtered['combined_tokens'].head())


0    [33025, 1335, 30, 512, 576, 270, 225, 72, 73, ...
1    [33025, 1335, 30, 323, 2254, 225, 72, 73, 1138...
2    [33025, 1335, 30, 1714, 71, 12639, 952, 538, 1...
3    [33025, 1335, 30, 323, 2254, 225, 72, 73, 643,...
4    [33025, 1335, 30, 20782, 225, 72, 73, 39829, 1...
Name: combined_tokens, dtype: object


### Check for empty tokens list

In [53]:
# Check for empty token lists in the 'combined_tokens' column
empty_tokens = df_filtered[df_filtered['combined_tokens'].apply(lambda x: len(x) == 0)]

# Print the number of rows with empty token lists
print(f"Number of rows with empty token lists: {len(empty_tokens)}")

# Optionally, display the first few rows with empty token lists
print(empty_tokens.head())


Number of rows with empty token lists: 0
Empty DataFrame
Columns: [corrected_title, corrected_ingredients, corrected_steps, combined, steps_word_count, ingredients_word_count, combined_word_count, combined_tokens, combined_tokens_length]
Index: []


In [54]:
# Get descriptive statistics for the 'combined_tokens' column
tokens_description = df_filtered['combined_tokens'].apply(len).describe()

# Print the descriptive statistics
print(tokens_description)


count    15793.000000
mean       301.999367
std        111.665435
min         36.000000
25%        215.000000
50%        304.000000
75%        390.000000
max        512.000000
Name: combined_tokens, dtype: float64


### Define padding token

In [55]:
# Manually set the padding token ID (using EOS token as pad token)
tokenizer.pad_token = tokenizer.eos_token  # Use end-of-sequence token as pad token
tokenizer.pad_token_id = tokenizer.encode(tokenizer.pad_token)[0]  # Get token ID for pad_token

# Check if the pad_token_id is set correctly
print(f"Pad Token ID: {tokenizer.pad_token_id}")

Pad Token ID: 50256


### Padding to max lenght

In [57]:
# Padding to max length
max_length = 512  # Define your desired maximum length

# Apply padding to the tokenized data
# Correct way to set the values using .loc to avoid SettingWithCopyWarning
df_filtered.loc[:, 'combined_tokens'] = df_filtered['combined_tokens'].apply(
    lambda x: x[:max_length] + [tokenizer.pad_token_id] * (max_length - len(x)) if len(x) < max_length else x[:max_length]
)


In [58]:
# Check the pad_token_id to ensure it's valid
print(f"Pad Token ID: {tokenizer.pad_token_id}")


Pad Token ID: 50256


### Check for invalid tokens

In [59]:
# Check for invalid token IDs (e.g., negative or zero values)
invalid_directions = df[df['combined_tokens'].apply(lambda x: any(i < 0 for i in x))]

print(f"Invalid combined tokens: {len(invalid_directions)}")



Invalid combined tokens: 0


### Check all values are lists

In [60]:
# Check that all values are lists
non_list_combined = df[~df['combined_tokens'].apply(lambda x: isinstance(x, list))]

print(f"Non-list combined tokens: {len(non_list_combined)}")

Non-list combined tokens: 0


### Check for non integer values

In [61]:
# Check for non-integer values in the token lists
invalid_combined_values = df[df['combined_tokens'].apply(lambda x: any(not isinstance(i, int) for i in x))]

print(f"Invalid combined token values: {len(invalid_combined_values)}")

Invalid combined token values: 0


### Check datatypes

In [62]:
# Check data types
print(df['combined_tokens'].apply(type).value_counts())

combined_tokens
<class 'list'>    19271
Name: count, dtype: int64


### Last inspection before Trainning

In [63]:
df["combined_tokens"].head(100)

,combined_tokens
0,"[33025, 1335, 30, 512, 576, 270, 225, 72, 73, ..."
1,"[33025, 1335, 30, 323, 2254, 225, 72, 73, 1138..."
2,"[33025, 1335, 30, 1714, 71, 12639, 952, 538, 1..."
3,"[33025, 1335, 30, 323, 2254, 225, 72, 73, 643,..."
4,"[33025, 1335, 30, 20782, 225, 72, 73, 39829, 1..."
...,...
96,"[33025, 1335, 30, 512, 576, 270, 225, 72, 73, ..."
97,"[33025, 1335, 30, 10306, 303, 9786, 225, 72, 7..."
98,"[33025, 1335, 30, 17960, 225, 72, 73, 11380, 2..."
99,"[33025, 1335, 30, 28784, 653, 2345, 320, 14479..."


In [65]:
from sklearn.model_selection import train_test_split


In [68]:
df

,corrected_title,corrected_ingredients,corrected_steps,combined,steps_word_count,ingredients_word_count,combined_word_count,combined_tokens,combined_tokens_length
0,arepas de queso,uno taza de harina de arepa blanca o amarilla ...,"combine la harina de maíz, agua caliente, el q...",Titulo: arepas de queso Ingredientes: uno taza...,92,29,127,"[33025, 1335, 30, 512, 576, 270, 225, 72, 73, ...",184
1,sudado de pollo,ocho muslos de pollo sin la piel uno cucharada...,"en una olla grande, caliente el aceite vegetal...",Titulo: sudado de pollo Ingredientes: ocho mus...,92,84,182,"[33025, 1335, 30, 323, 2254, 225, 72, 73, 1138...",270
2,sancocho trifásico,uno taza de cebolla picada uno pimientón rojo ...,"coloque la cebolla, el pimientón, el ajo y el ...",Titulo: sancocho trifásico Ingredientes: uno t...,103,114,222,"[33025, 1335, 30, 1714, 71, 12639, 952, 538, 1...",336
3,sudado de posta o muchacho,"uno posta o muchacho de tres a cuatro libras, ...","en una olla grande ponga la carne, el caldo, e...",Titulo: sudado de posta o muchacho Ingrediente...,85,72,165,"[33025, 1335, 30, 323, 2254, 225, 72, 73, 643,...",240
4,sopa de lentejas colombiana,uno cucharadita de aceite vegetal uno taza de ...,cocinar el chorizo en el aceite en una olla gr...,Titulo: sopa de lentejas colombiana Ingredient...,129,74,210,"[33025, 1335, 30, 20782, 225, 72, 73, 39829, 1...",295
...,...,...,...,...,...,...,...,...,...
28233,berenjenas rellenas de carne picada y bechamel...,cuatro berenjenas dos zanahorias uno cebolla u...,uno empezaremos a preparar estas berenjenas re...,Titulo: berenjenas rellenas de carne picada y ...,388,49,449,"[33025, 1335, 30, 33220, 992, 16993, 992, 225,...",655
28234,puré de patatas con queso y huevo,dos patatas dos huevos cien gramos de parmesan...,"uno cocemos las patatas y, una vez listas, las...",Titulo: puré de patatas con queso y huevo Ingr...,226,21,257,"[33025, 1335, 30, 32343, 225, 72, 73, 15281, 3...",345
28235,rollitos de calabacín rellenos de jamón y queso,un medio calabacín cien gramos de jamón cocido...,uno lo primero que debemos hacer es lavar bien...,Titulo: rollitos de calabacín rellenos de jamó...,231,15,257,"[33025, 1335, 30, 26195, 1284, 225, 72, 73, 48...",370
28236,calabacín relleno de quinoa,cuatro calabacines uno cebolla uno pimiento ve...,uno lo primero que tenemos que hacer para elab...,Titulo: calabacín relleno de quinoa Ingredient...,371,47,425,"[33025, 1335, 30, 48869, 15105, 225, 72, 73, 4...",616


In [70]:
# Split into training and validation (80% train, 20% validation, for example)
train_df, valid_df = train_test_split(df_filtered, test_size=0.2, random_state=42)

# Now both train_df and valid_df are ready with tokenized, padded, and filtered data
print(f"Training set size: {train_df.shape[0]}")
print(f"Validation set size: {valid_df.shape[0]}")


Training set size: 12634
Validation set size: 3159


In [72]:
train_df.head(100)

,corrected_title,corrected_ingredients,corrected_steps,combined,steps_word_count,ingredients_word_count,combined_word_count,combined_tokens,combined_tokens_length
15773,fresca ensalada de endibias,ciento cincuenta ml de aceite de oliva virgen ...,uno primero en un bol preparamos una vinagreta...,Titulo: fresca ensalada de endibias Ingredient...,115,35,157,"[33025, 1335, 30, 14104, 17960, 225, 72, 73, 4...",240
15002,ponche chevalier,el jugo de doce limones uno taza de azúcar en ...,uno en un recipiente grande mezcle desde el ju...,Titulo: ponche chevalier Ingredientes: el jugo...,42,41,88,"[33025, 1335, 30, 1351, 1211, 809, 38758, 392,...",151
8993,pastela moruna con thermomix,uno paquete de masa de hojaldre congelado quin...,uno el primer paso para preparara esta receta ...,Titulo: pastela moruna con thermomix Ingredien...,208,64,279,"[33025, 1335, 30, 9320, 1763, 3211, 2412, 303,...",415
22000,vasitos de queso y fresa,trescientos cincuenta gramos de queso crema ti...,uno lo primero que tenemos que hacer para prep...,Titulo: vasitos de queso y fresa Ingredientes:...,292,38,338,"[33025, 1335, 30, 2933, 1284, 225, 72, 73, 928...",495
24442,crema pastelera con thermomix,dos unidades de huevos setenta y cinco gramos ...,uno pon la mariposa en la máquina y agrega tod...,Titulo: crema pastelera con thermomix Ingredie...,82,29,118,"[33025, 1335, 30, 8246, 11842, 422, 303, 7472,...",170
...,...,...,...,...,...,...,...,...,...
16195,guiso de beterraga,uno atado(s) betarraga(s) dos cebolla(s) chica...,uno cuesa las beterragas y luego paselas por e...,Titulo: guiso de beterraga Ingredientes: uno a...,42,24,72,"[33025, 1335, 30, 48725, 225, 72, 73, 337, 346...",129
24945,mojo de tomates asados (isla de fuerteventura),dos tomates cuatro ó cinco dientes de ajos com...,"uno se machacan los ajos, los cominos, la sal ...",Titulo: mojo de tomates asados (isla de fuerte...,51,14,75,"[33025, 1335, 30, 829, 686, 225, 72, 73, 23612...",113
9700,arroz a la argentina,cuatro cucharadas de aceite ciento cincuenta g...,uno se fríe la cebolla en aceite hasta que se ...,Titulo: arroz a la argentina Ingredientes: cua...,59,27,93,"[33025, 1335, 30, 9635, 267, 280, 9230, 49427,...",125
7680,burritos mexicanos de carne picada,dos cucharadas soperas de aceite ciento cincue...,uno para la base de nuestros burritos mexicano...,Titulo: burritos mexicanos de carne picada Ing...,202,50,260,"[33025, 1335, 30, 4664, 4973, 9823, 225, 72, 7...",356


In [73]:
valid_df.head(100)

,corrected_title,corrected_ingredients,corrected_steps,combined,steps_word_count,ingredients_word_count,combined_word_count,combined_tokens,combined_tokens_length
24651,arroz con dulce,uno coco uno taza de arroz dos trozos grandes ...,uno saque la masa del coco. dos rállela. tres ...,Titulo: arroz con dulce Ingredientes: uno coco...,78,19,103,"[33025, 1335, 30, 9635, 303, 7653, 49427, 30, ...",165
418,almojábanas colombianas,uno un cuarto taza masarepa o harina de maíz b...,precaliente el horno a cuatrocientos ° f. mezc...,Titulo: almojábanas colombianas Ingredientes: ...,58,41,104,"[33025, 1335, 30, 333, 477, 78, 1865, 1216, 28...",148
13966,albondigón colombiano casero,uno libra de carne de res uno libra de carne d...,uno precalentar el horno a 350°f o 220c. en un...,Titulo: albondigón colombiano casero Ingredien...,142,54,202,"[33025, 1335, 30, 8710, 2191, 13857, 10034, 17...",309
13263,ternera en adobo,uno kg. de ternera en trozos cinco chiles anch...,"uno para marinar la carne, déjela reposar dura...",Titulo: ternera en adobo Ingredientes: uno kg....,131,43,180,"[33025, 1335, 30, 37573, 225, 73, 82, 2467, 16...",272
8088,palomitas de queso,cien gramos de queso mozarella cien gramos de ...,uno para comenzar con la receta de palomitas d...,Titulo: palomitas de queso Ingredientes: cien ...,213,50,269,"[33025, 1335, 30, 47850, 225, 72, 73, 9288, 49...",393
...,...,...,...,...,...,...,...,...,...
22400,crepas de fresa con queso crema,uno taza de harina de trigo (140 gramos) uno u...,uno estas crepas dulces con fruta son muy fáci...,Titulo: crepas de fresa con queso crema Ingred...,221,54,284,"[33025, 1335, 30, 630, 2974, 225, 72, 73, 3358...",396
19753,pastas con salsa de aguacate,ciento cincuenta gramos de pasta un medio unid...,"uno antes de realizar estas pastas saludables,...",Titulo: pastas con salsa de aguacate Ingredien...,146,40,194,"[33025, 1335, 30, 32548, 303, 9786, 225, 72, 7...",306
13162,magro de cerdo con cominos y cilantro,quinientos gr magro de cerdo en dados dos cuch...,uno freír el magro en aceite de oliva hasta qu...,Titulo: magro de cerdo con cominos y cilantro ...,43,21,74,"[33025, 1335, 30, 2558, 302, 225, 72, 73, 1733...",112
9478,sándwich de jamón y queso con huevo,uno trozo de queso amarillo dos lonchas de jam...,uno lo primero que haremos para hacer este del...,Titulo: sándwich de jamón y queso con huevo In...,294,32,336,"[33025, 1335, 30, 32641, 23272, 225, 72, 73, 1...",473


#3) Trainning Phase

In [74]:
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from torch.optim import AdamW
from tqdm import tqdm

In [75]:
# Define a simple Dataset class to handle tokenized inputs
class RecipeDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.input_ids = df['combined_tokens'].tolist()
        self.attention_mask = [[1] * len(ids) for ids in self.input_ids]  # Attention mask is 1 for all tokens

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long)
        }

In [76]:
# Prepare the DataLoaders
train_dataset = RecipeDataset(train_df)
valid_dataset = RecipeDataset(valid_df)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=8)

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("flax-community/gpt-2-spanish")

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [77]:
# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
# Save model and optimizer
model_save_path = r"F:\New folder (3)\AAAA Segundo SEMESTRE 2024\Analitica textual con ML\ProyectoFinal\gpt2_spanish_model_epoch_{}.pt"
optimizer_save_path = r"F:\New folder (3)\AAAA Segundo SEMESTRE 2024\Analitica textual con ML\ProyectoFinal\optimizer_epoch_{}.pt"

epochs = 3

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}", ncols=100):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update model weights
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Average Training Loss: {avg_train_loss}")

    # Validation step
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(valid_dataloader, desc=f"Validating Epoch {epoch+1}", ncols=100):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            val_loss = outputs.loss
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(valid_dataloader)
    print(f"Epoch {epoch+1} - Average Validation Loss: {avg_val_loss}")

    # Save model and optimizer after each epoch
    torch.save(model.state_dict(), model_save_path.format(epoch+1))
    torch.save(optimizer.state_dict(), optimizer_save_path.format(epoch+1))


Training Epoch 1:   0%|                                         | 2/1580 [02:51<37:23:21, 85.30s/it]